In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import category_encoders as ce
from sklearn import preprocessing

## 3. Encode cat cols

In [2]:
dt_all_encoded = pd.read_csv("../../data/Mercedes_Benz_Greener_Manufacturing/data/dt_all_cleaned.csv")

In [3]:
# cols_cat
cols_cat = dt_all_encoded.drop("IsTrainTest", axis = 1).select_dtypes(include = ['object']).columns.values

In [4]:
dt_all_encoded.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X377,X378,X379,X380,X383,X384,IsTrainTest,IsDupRow_All,IsDupRow_Cat,IsDupRow_Int
0,0,130.81,k,v,at,a,d,u,j,o,...,1,0,0,0,0,0,train,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,0,0,0,0,0,0,train,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,train,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,train,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,train,0,0,0


### 3.1 One-Hot Encoding

In [ ]:
dt_cat_onehot = pd.get_dummies(dt_all_encoded[cols_cat])
dict_ohe = {x: "Encode_ohe_" + x for x in dt_cat_onehot.columns.values}
dt_cat_onehot = dt_cat_onehot.rename(columns = dict_ohe)

### 3.2 TargetMean Encoding

In [ ]:
# oof to encode cols_cat with TargetMean
def getTargetMean(dt, dt_all, cols, k = 3):

    # init dt_targetMean
    dt_targetMean = pd.DataFrame()

    for col in cols:
    
        # init dt_targetMean_oof
        dt_targetMean_oof = pd.DataFrame()
        
        # X_targetMean_Kfold, y_targetMean_Kfold
        X_targetMean_Kfold = dt[[col, "y"]]
        y_targetMean_Kfold = dt[col].values

        # oof cv
        skf = StratifiedKFold(n_splits = k)
        
        
        for i, (ind_in, ind_out) in enumerate(skf.split(X_targetMean_Kfold, y_targetMean_Kfold)):


            # init dt_targetMean_oof
            dt_targetMean_val = pd.DataFrame()

            # X_in, X_out, y_in, y_out
            X_in, X_out = X_targetMean_Kfold.iloc[ind_in], X_targetMean_Kfold.iloc[ind_out]
            y_in, y_out = y_targetMean_Kfold[ind_in], y_targetMean_Kfold[ind_out]

            # calc TargetMean
            for val in set(X_in[col].values):
                dt_targetMean_temp = pd.DataFrame({"Value": [val]
                                                   , "TargetMean_" + str(i): [X_in.loc[X_in[col] == val].y.mean()]})
                dt_targetMean_val = pd.concat([dt_targetMean_val, dt_targetMean_temp])

            # merge with oof
            if i == 0:
                dt_targetMean_oof = pd.merge(X_targetMean_Kfold.drop("y", axis = 1).drop_duplicates(), dt_targetMean_val
                                             , how = "left", left_on = col, right_on = "Value")
                dt_targetMean_oof = dt_targetMean_oof.drop(col, axis = 1)
            else:
                dt_targetMean_oof = pd.merge(dt_targetMean_oof, dt_targetMean_val
                                             , how = "left", on = "Value")

        # move Value to the first column
        value = dt_targetMean_oof['Value']
        dt_targetMean_oof.drop(labels = ["Value"], axis = 1,inplace = True)
        dt_targetMean_oof.insert(0, 'Value', value)
        # assign col
        dt_targetMean_oof.insert(0, 'Col', col)
        
        # concat with col
        dt_targetMean = pd.concat([dt_targetMean, dt_targetMean_oof])
    
    # mean of oof
    dt_targetMean["TargetMean"] = dt_targetMean.filter(regex = "TargetMean").mean(axis = 1)
    # fill zero
    dt_targetMean = dt_targetMean.fillna(0)
    # remove oof cols
    dt_targetMean = dt_targetMean[["Col", "Value", "TargetMean"]]

    # merge to original table
    dt_cat_cols = dt_all[cols]
    
    for col in cols_cat:
        dt_cat_cols = pd.merge(dt_cat_cols, dt_targetMean.loc[dt_targetMean["Col"] == col]
                           , how = "left", left_on = col, right_on = "Value")
        dt_cat_cols = dt_cat_cols.drop(["Value", "Col"], axis = 1)
        dt_cat_cols = dt_cat_cols.rename(columns = {"TargetMean": "Encode_TargetMean_" + col})
    # fill zero
    dt_cat_cols = dt_cat_cols.fillna(0)
    
    # only select targetMean cols
    dt_cat_targetMean = dt_cat_cols.filter(regex = "Encode_TargetMean_")
        
    return dt_cat_targetMean

In [ ]:
dt_cat_targetMean = getTargetMean(dt_all_encoded.loc[dt_all_encoded["IsTrainTest"] == "train"]
                                  , dt_all_encoded
                                  , cols_cat
                                  , 3)

### 3.3 Frequency Encoding

In [ ]:
def getFrequency(dt, cols):
    # calc the cols freq
    dt_cat_freq = pd.DataFrame()
    for col in cols:
        dt_col_freq_temp = pd.DataFrame({"Col": col
                                         , "Value": dt[col].value_counts().index.values
                                         , "Freq": dt[col].value_counts().values})

        dt_cat_freq = pd.concat([dt_cat_freq, dt_col_freq_temp])

    # merge to origin
    dt_cat_cols = dt_all_encoded[cols]
    for col in cols:
        dt_cat_cols = pd.merge(dt_cat_cols, dt_cat_freq.loc[dt_cat_freq["Col"] == col]
                           , how = "left", left_on = col, right_on = "Value")
        dt_cat_cols = dt_cat_cols.drop(["Value", "Col"], axis = 1)
        dt_cat_cols = dt_cat_cols.rename(columns = {"Freq": "Encode_Freq_" + col})

    dt_cat_cols = dt_cat_cols.filter(regex = "Encode_Freq_")
    
    return dt_cat_cols

In [ ]:
dt_cat_freq = getFrequency(dt_all_encoded, cols_cat)

### 3.4 Binary

In [ ]:
encode_binary = ce.BinaryEncoder(cols_cat)
dt_cat_binary = encode_binary.fit_transform(dt_all_encoded[cols_cat])
dt_cat_binary.columns = "Encode_Binary_" + dt_cat_binary.columns

### 3.5 Ordinal X0, X5

In [ ]:
def getOrdinal(dt, col):
    dt_ordinal = pd.DataFrame()
    for val in set(dt[col].values):
        dt_ordinal_temp = pd.DataFrame({"Value": [val]
                                        , "Encode_Ordinal_" + col: dt.loc[dt[col] == val].y.mean()})
        dt_ordinal = pd.concat([dt_ordinal, dt_ordinal_temp])

    dt_cat_ordinal = pd.merge(dt[col].to_frame(), dt_ordinal
                              , how = "left", left_on = col, right_on = "Value")
    dt_cat_ordinal = dt_cat_ordinal.drop([col, "Value"], axis = 1)
    return dt_cat_ordinal

In [ ]:
dt_cat_ordinal_X0 = getOrdinal(dt_all_encoded, "X0")
dt_cat_ordinal_X5 = getOrdinal(dt_all_encoded, "X5")

### 3.6 Label

In [5]:
encode_label = ce.OrdinalEncoder(cols_cat)
dt_cat_label = encode_label.fit_transform(dt_all_encoded[cols_cat])
dt_cat_label.columns = "Encode_Binary" + dt_cat_label.columns

/home/noahhhhhh/Env/deepNoah/lib/python3.5/site-packages/category_encoders/ordinal.py:178: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  X[col] = X[col].astype(int).reshape(-1, )
/home/noahhhhhh/Env/deepNoah/lib/python3.5/site-packages/category_encoders/ordinal.py:167: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  X[switch.get('col')] = X[switch.get('col')].astype(int).reshape(-1, )


In [7]:
dt_all_encoded[cols_cat].head()

,X0,X1,X2,X3,X4,X5,X6,X8
0,k,v,at,a,d,u,j,o
1,k,t,av,e,d,y,l,o
2,az,w,n,c,d,x,j,x
3,az,t,n,f,d,x,l,e
4,az,v,n,f,d,h,d,n


In [8]:
dt_cat_label.head()

,Encode_BinaryX0,Encode_BinaryX1,Encode_BinaryX2,Encode_BinaryX3,Encode_BinaryX4,Encode_BinaryX5,Encode_BinaryX6,Encode_BinaryX8
0,36,5,44,2,2,6,2,13
1,36,24,39,5,2,29,0,13
2,41,18,41,0,2,2,2,8
3,41,24,41,4,2,2,0,19
4,41,5,41,4,2,5,3,12


### 3.7 Combine all encoders

In [ ]:
# OHE
dt_all_encoded = dt_all_encoded.drop(cols_cat, axis = 1).join(dt_cat_onehot)
dt_all_encoded.shape

In [ ]:
# TargetMean
dt_all_encoded = dt_all_encoded.join(dt_cat_targetMean)
dt_all_encoded.shape

In [ ]:
# Frequency
dt_all_encoded = dt_all_encoded.join(dt_cat_freq)
dt_all_encoded.shape

In [ ]:
# Binary
dt_all_encoded = dt_all_encoded.join(dt_cat_binary)
dt_all_encoded.shape

In [ ]:
# Ordinal X0
dt_all_encoded = dt_all_encoded.join(dt_cat_ordinal_X0)
dt_all_encoded.shape

In [ ]:
# Ordinal X5
dt_all_encoded = dt_all_encoded.join(dt_cat_ordinal_X5)
dt_all_encoded.shape

In [ ]:
# Label
dt_all_encoded = dt_all_encoded.join(dt_cat_label)
dt_all_encoded.shape

### 3.7 Save dt_all_encoded

In [ ]:
dt_all_encoded.to_csv("../../data/Mercedes_Benz_Greener_Manufacturing/data/dt_all_encoded.csv", index = False)